In [37]:
def toLower(text):
    ''' Convert text to lower case'''
    return text.lower()
#toLower(df.loc[1,'title'])
################################################################
from decimal import Decimal
import re
import inflect


p = inflect.engine()
reg = r'([0-9]+)'

def isFLoat(strNum):
    '''converte float number to word'''
    try:
        float(strNum)
        return True
    except:
        return False
##########################################################

def convertNumbers(text):
    ''' Convert texnumbers to words'''
    tempText = text.split()
    newText = []
    for word in tempText:
        tempList = re.split(reg,word)
        for miniWord in tempList:
            if miniWord.isdigit() or isFLoat(miniWord):
                temp = p.number_to_words(miniWord)
                newText.append(removePunctuation(temp))
            else:
                newText.append(miniWord)        
    tempText = ' '.join(newText)
    return tempText
##################################################################
def removeWhiteSpace(text):
    '''remove whitespace from text'''
    return " ".join(text.split())
#removeWhiteSpace("hi   hi")
######################################################################
import string
translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
def removePunctuation(text):
    ''' remove punctuation from text'''
    global translator
    return text.translate(translator)
#removePunctuation(df.loc[1,'title'])
##############################################################

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

def removeStopWords(text):
    ''' remove stopwords from text'''
    sw = set(stopwords.words("english"))
    wt = word_tokenize(text)
    filteredText = [word for word in wt if word not in sw]
    return ' '.join(filteredText)
#print(df.loc[1,'title'])
#print(removeStopWords(df.loc[0,'title']))
###############################################################

from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()
def stemmingWords(text):
    ''' stemm words'''
    global stemmer
    wt = word_tokenize(text)
    stems = []
    for word in wt:
        temp = stemmer.stem(word)
        stems.append(temp)
    return ' '.join(stems)
#print(stemmingWords(df.loc[0,'title']))
###############################################################
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import pos_tag, defaultdict

lemmatizer = WordNetLemmatizer()

tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemmatizeWords(text):
    ''' lemmatize words'''
    tokens = word_tokenize(text)
    lmtzr = WordNetLemmatizer()
    lemmas = [lmtzr.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(tokens) ]
    return ' '.join(lemmas)
#print(lemmatizeWords(df.loc[0,'title']))
################################################################

In [38]:
import pandas as pd
def titlePreProcesse(text):
    
    tempText = toLower(text)
    tempText = removePunctuation(tempText)
    tempText = convertNumbers(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText
#print(titlePreProcesse(df.loc[0,'title']))
###########################################################

def preprocessedAntiqueData(dataFrame:pd.DataFrame):
    tempFrame = pd.DataFrame(columns = ['id', 'title'])
    tempFrame1 = pd.DataFrame(columns = ['id', 'title'])
    tempFrame2 = pd.DataFrame(columns = ['id', 'title'])
    tempFrame3 = pd.DataFrame(columns = ['id', 'title'])
    tempFrame4 = pd.DataFrame(columns = ['id', 'title'])

    for i in dataFrame.index[0:100000:1]:
        try:
            tempT = titlePreProcesse(dataFrame.loc[i, 'title'])
            dataf=pd.DataFrame([[dataFrame.loc[i,'id'],tempT]],columns=['id','title'])
            tempFrame1 = pd.concat([tempFrame1,dataf], ignore_index=True)
        except:
            print(i,dataFrame.loc[i, 'title'])
            continue
            #raise 

    for i in dataFrame.index[100000:200000:1]:
        try:
            tempT = titlePreProcesse(dataFrame.loc[i, 'title'])
            dataf=pd.DataFrame([[dataFrame.loc[i,'id'],tempT]],columns=['id','title'])
            tempFrame2 = pd.concat([tempFrame2,dataf], ignore_index=True)
        except:
            print(i,dataFrame.loc[i, 'title'])
            continue
            #raise  
      
    for i in dataFrame.index[200000:300000:1]:
        try:
            tempT = titlePreProcesse(dataFrame.loc[i, 'title'])
            dataf=pd.DataFrame([[dataFrame.loc[i,'id'],tempT]],columns=['id','title'])
            tempFrame3 = pd.concat([tempFrame3,dataf], ignore_index=True)
        except:
            print(i,dataFrame.loc[i, 'title'])
            continue
            #raise  
      
    for i in dataFrame.index[300000::1]:
        try:
            tempT = titlePreProcesse(dataFrame.loc[i, 'title'])
            dataf=pd.DataFrame([[dataFrame.loc[i,'id'],tempT]],columns=['id','title'])
            tempFrame4 = pd.concat([tempFrame4,dataf], ignore_index=True)
        except:
            print(i,dataFrame.loc[i, 'title'])
            continue
            #raise   
    
    tempFrame=pd.concat([tempFrame1,tempFrame2,tempFrame3,tempFrame4],ignore_index=True)
    tempFrame.fillna('', inplace=True)
    return tempFrame

In [39]:
import pandas as pd
def QueryTitlePreProcesse(a):
    tempText = toLower(a)
    tempText = removePunctuation(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemmingWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText
########################################################
def preprocesseQuery(dataFrame:pd.DataFrame):
    tempFrame = pd.DataFrame(columns = ['id', 'title'])
    for i in dataFrame.index:
        try:
            tempTilte=queryTitlePreProcesse(dataFrame.loc[i,'title'])
            tempFrame.loc[len(tempFrame.index)] = [dataFrame.loc[i,'id'],tempTitle]
        except:
            print(i)
            continue
            #raise 
    
    tempFrame.fillna('', inplace=True)
    return tempFrame


In [40]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import  cosine_similarity
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
########################################################################

transformer = None
tfidfTable  = None
def initializeTfidfTable(data: pd.DataFrame):
    global transformer, tfidfTable
    transformer = FeatureUnion([('title_tfidf', 
                      Pipeline([
                        ('extract_field',
                                  FunctionTransformer(lambda x: x['title'], 
                                                      validate=False)),
                                ('tfidf', 
                                  TfidfVectorizer(norm='l1' ,ngram_range=(1,2)))]))                              
                      
    ])
    tfidfTable = transformer.fit_transform(data)

In [41]:
import spacy
from nltk.tokenize import word_tokenize

words:list=[]
def readWords(data:pd.DataFrame):
    global words
    for id in data.index[1:]:
        tempWords=[]
        for word in data.loc[id,'title'].split():
            tempWords.append(word)
        words.append(tempWords)
#readWords(processedData)

#########################################################
import gensim
from gensim.models import Word2Vec

model=None
def createModel(words:list):
    global model
    model=gensim.models.Word2Vec(words,min_count=1,vector_size=300,window=5)
    model.train(words, total_examples=len(words), epochs=10)
    model.save('WEmodel_final_training_words.bin')
    print("model is created")
#createModel(words)


In [43]:
import pandas as pd

vectors:list=[]
    
def makeVectors(dataFrame:pd.DataFrame()):
    global vectors
    try:
        for id in dataFrame.index:
            avgword2vec =None
            count=0
            tempTitle=dataFrame.loc[id,'title']
            for word in tempTitle.split():
                if word in model.wv:
                    count+=1
                    if avgword2vec is None:
                        avgword2vec  =model.wv[word]
                    else:
                        avgword2vec =avgword2vec + model.wv[word]

            if avgword2vec is not None:
                avgword2vec = avgword2vec / count

                vectors.append(avgword2vec)
            else:
                vectors.append([0]*300) 
        #print(len(te), len(vectors))
        
    except:
        print("error")
#makeVectors(processedData)
##################################################################
def makeVectorQuery(query:str):
    try:
        avg=None
        count=0
        for word in query.split():
            if word in model.wv:
                count+=1
                if avg is None:
                    avg=model.wv[word]
                else:
                    avg=avg+model.wv[word]
        if avg is not None:
            avg=avg/count
            return [avg]
    except:
        print('error')



In [44]:
from sklearn.metrics.pairwise import  cosine_similarity

def matching(query):
    global transformer, tfidfTable
    querytfidf = transformer.transform(query)
    cosine_sim=cosine_similarity(querytfidf,tfidfTable).flatten()
    return cosine_sim
    
#################################################################
import numpy as np
def matchingWE(query)->np.ndarray:
    global vectors
    vecQuery=makeVectorQuery(query)
    cosine_sim=cosine_similarity(vecQuery,vectors).flatten()
    return cosine_sim

In [45]:
def preprocesseSearchInput(query) -> pd.DataFrame:
    
    #data = dataDic.get('query')
    tempI = 1
    tempW = QueryTitlePreProcesse(query)
    tempFrame = pd.DataFrame(columns = ['id', 'title'])
    tempFrame.loc[len(tempFrame.index)] = [tempI,tempW]
    tempFrame.fillna('', inplace=True)
    return tempFrame

In [46]:
def absSub(a,b):
    return abs(a-b)
######################################################
def search(qDF:pd.DataFrame,data:pd.DataFrame, n):
    ''' search for input and return list of ids of the result'''
    try:
        resultlis = []
        similars = matching(qDF)
        tempIds = similars.argsort(axis=0)[-n:][::-1]
        tempFrame = pd.DataFrame(columns = ['id', 'title'])
        for i in tempIds:
            tempFrame.loc[len(tempFrame.index)] = [data.loc[i,'id'],data.loc[i,"title"]]
        return tempFrame
    except:
        print("false")
        raise
##########################################################
def searchWE(qDF:pd.DataFrame,data:pd.DataFrame, n):
    ''' search for input and return list of ids of the result'''
    try:
        resultlis = []
        similars = matchingWE(qDF.loc[0,'title'])
        tempIds = similars.argsort(axis=0)[-n:][::-1]
        tempFrame = pd.DataFrame(columns = ['id', 'title'])
        for i in tempIds:
            tempFrame.loc[len(tempFrame.index)] = [data.loc[i,'id'],data.loc[i,"title"]]
        return tempFrame
    except:
        print("false")
        raise

In [47]:
def searchInput(typeSearch,query):
    dataPD: pd.DataFrame = preprocesseSearchInput(query)
    if  typeSearch==2:
        resultIds = searchWE(dataPD, dataFrame, 10)
    else:
        resultIds = search(dataPD, dataFrame, 10) 
    return resultIds

In [48]:
from sklearn.metrics import precision_score, recall_score
def reSizeLists(l1:list, l2:list, n :int):
    '''resize lists to have the same len'''
    
    if len(l1) < len(l2):
        l2 = l2[0:len(l1)]
    while len(l1) > len(l2):
        l1 = l1[0:len(l2)]
    if len(l1)<n:
        n=len(l1)
    return l1[0:n], l2[0:n]

def getQrelsArray( qrelsData: pd.DataFrame,qid):
    qrelsArray:list = []
    for index in qrelsData.index:
        if qrelsData.loc[index,'query_id']==qid:
            qrelsArray.append(qrelsData.loc[index,'relevance'])
    return qrelsArray

def getRes( resData: pd.DataFrame,qid):
    resArray:list = []
    for index in resData.index[1::]:
        if resData.loc[index,'qid']==qid:
            resArray.append(resData.loc[index,'rid'])
    return resArray

def precisionAtK(resData:pd.DataFrame, qrelsData: pd.DataFrame,n):
    precisionsAtK:list = []
    for i in resData.index[1::10]:
        resArray:list = []
        x=i+10 
        for inde in range(int(i),int(x)):
            resArray.append(resData.loc[inde,'rid'])
        #resArray=getRes(resData,resData.loc[i,'qid'])
        qrelsArray=getQrelsArray(qrelsData,resData.loc[i,'qid'])
        if len(qrelsArray) == 0: 
            continue
        #reSet=set(resArray)
        #qrSet=set(qrelsArray)
        #precisionsAtK.append(len(reSet&qrSet)/len(reSet))
        resArray, qrelsArray = reSizeLists(resArray, qrelsArray,n)
        precisionsAtK.append(precision_score(qrelsArray, resArray, average='micro'))
    return sum(precisionsAtK) / len(precisionsAtK)


def calPrecisionAtK(querData:pd.DataFrame,resData:pd.DataFrame, qrelsData: pd.DataFrame,n):
    precisionsAtK:list = []
    for i in querData.index[1::]:
        resArray:list = []
        resArray=getRes(resData,querData.loc[i,'query_id'])
        
        if len(resArray) == 0 : 
            print("res=0")
            continue
        qrelsArray=getQrelsArray(qrelsData,querData.loc[i,'query_id'])
        if len(qrelsArray) == 0 : 
            continue
        
        #resArray, qrelsArray = reSizeLists(resArray, qrelsArray,n)
        reSet=set(resArray)
        qrSet=set(qrelsArray)
        precisionsAtK.append((reSet&qrSet)/len(reSet))
        #precisionsAtK.append(precision_score(qrelsArray, resArray, average='micro'))
    return sum(precisionsAtK) / len(precisionsAtK)


def recallsAtK(resData:pd.DataFrame, qrelsData: pd.DataFrame,n):
    recallsAtK:list = []
    for i in resData.index[1::10]:
        resArray:list = []
        for inde in range(int(i),int(i+10)):
            resArray.append(resData.loc[inde,'rid'])
        qrelsArray=getQrelsArray(qrelsData,resData.loc[i,'qid'])
        if len(qrelsArray) == 0: 
            continue
        resArray, qrelsArray = reSizeLists(resArray, qrelsArray,n)
        recallsAtK.append(recall_score(qrelsArray, resArray, average='micro'))
    return sum(recallsAtK) / len(recallsAtK)


def MAPAtK(resData:pd.DataFrame, qrelsData: pd.DataFrame):
    APAtk:list=[]
    for i in resData.index[1::10]:
        resArray:list = []   
        for inde in range(int(i),int(i)+10):
            resArray.append(resData.loc[inde,'rid'])
        qrelsArray=getQrelsArray(qrelsData,resData.loc[i,'qid'])
        if len(qrelsArray) == 0: 
            continue
        resArray, qrelsArray = reSizeLists(resArray, qrelsArray,n)
        for id in range(len(resArray)):
            tempresArray, tempqrelsArray = reSizeLists(resArray, qrelsArray,id)
            APAtk.append(precision_score(tempqrelsArray,tempresArray,average='micro'))
    return sum(APAtk) / len(APAtk)

def MRR(resData:pd.DataFrame,qrelsData: pd.DataFrame):
    mrr:list=[]
    for i in resData.index[1::10]:
        resArray:list = []
        for inde in range(int(i),int(i)+10):
            resArray.append(resData.loc[inde,'rid'])
        qrelsArray=getQrelsArray(qrelsData,resData.loc[i,'qid'])
        
        for id in range(len(resArray)):
            if resArray[id] in qrelsArray:
                mrr.appent(1/(id+1))
                break
    
    return sum(mrr)/len(mrr)  

In [49]:
def executionQuery(qDataFrame:pd.DataFrame,data:pd.DataFrame, n,typeSearch):
    ''' search for all queries in the queries file and get the most n similar document .I'''
    result = pd.DataFrame(columns=['qid','rid','rank'])
    
    for i in qDataFrame.index[1:250]:
        tempFrame =pd.DataFrame(columns=['qid','rid','rank'])
        try:
            resultDict:dict = {}
            #tempIds:list = matching(pd.DataFrame(qDataFrame.loc[qDataFrame.index == i,:]))
            tempW = QueryTitlePreProcesse(qDataFrame.loc[i,'title'])
            tempFrame1= pd.DataFrame(columns = ['id', 'title'])
            tempFrame1.loc[len(tempFrame.index)] = [1,tempW]
            
            #query=QueryTitlePreProcesse(qDataFrame.loc[i,'title'])
            if typeSearch==1:
                tempIds:list = matchingWE(tempFrame1)
            else:
                tempIds:list = matchingWE(tempW)
            tempIds = tempIds.argsort(axis=0)[-n:][::-1] 
            tempList = []
            qid=qDataFrame.loc[i,'query_id']
            for id in tempIds[0:n:]:
                #dataf=pd.DataFrame([qid,id])
                tempFrame.loc[len(tempFrame.index)] = [qid,data.loc[id,'id'],id]
            result = pd.concat([result,tempFrame], ignore_index=True)
                
        except:
            print(i)
            #raise
    return result


In [50]:
import pandas as pd
def readdata():
    qDataFrame:pd.DataFrame = pd.read_table('antique/collection.tsv')
    qDataFrame.fillna('', inplace=True)
    return qDataFrame
###################################################################

def readPPData():
    qDataFrame:pd.DataFrame = pd.read_table('antique/preprocessData.tsv')
    qDataFrame.fillna('', inplace=True)
    return qDataFrame
###################################################################

def readQueryFile():
    qDataFrame:pd.DataFrame = pd.read_table('antique/train/queries.txt',names=["query_id","title"])
    qDataFrame.fillna('', inplace=True)
    return qDataFrame
###################################################################

def readQrelsFile():
    qDataFrame:pd.DataFrame = pd.read_table('antique/train/qrels.tsv',names=['query_id','doc_id', 'relevance','iteration'],delim_whitespace=True)
    qDataFrame.fillna('', inplace=True)
    return qDataFrame


In [51]:

def readFiles():
    global dataFrame
    dataFrame=readdata()
    PPdata=readPPData()
    queriesDataFrame=readQueryFile()
    qrelsDataFrame=readQrelsFile()


In [52]:
def initvectors():
    global model
    #readWords(PPdata)
    #createModel(words)
    model = Word2Vec.load('WEmodel_final_training_words.bin')
    makeVectors(PPdata)


In [53]:
import pandas as pd
dataFrame:pd.DataFrame
def init():
    global dataFrame
    readFiles()
    initializeTfidfTable(dataFrame)
    initvectors()

In [54]:

d = {
    "query":"world",
    "type":2
}
#searchInput(2,"world")

In [55]:

def execution():
    re=executionQuery(queriesDataFrame,PPdata,10,2)
    re.to_csv('antique/train/resultWE.tsv',sep="\t")
    print("finished")

In [56]:
import pandas as pd
def evav():
    n=10

    re:pd.DataFrame = pd.read_table('antique/train/resultWE.tsv',names=['qid','rid','rank'])

    precisionAtK = precisionAtK(re, qrelsDataFrame,n)
    print(f'precacmon@{n} : {precisionAtK}')
    recallsAtK = recallsAtK(re, qrelsDataFrame,n)
    print(f'recallsAt@{n} : {recallsAtK}')
    MAPAtK =MAPAtK(re, qrelsDataFrame)
    print(f'MAPAt : {MAPAtK}')
    MRR = MRR(re, qrelsDataFrame)
    print(f'MRR : {MRR}')

In [57]:
from flask import Flask, request, jsonify
from importlib_metadata import method_cache
port = 8000
app = Flask(__name__)

@app.route('/', methods=['GET'])
def initialize():
    print("initialize")
    try:
        init()
        return jsonify(success=True)
    except:
        return jsonify(success=False)



@app.route('/search/<int:typeSearch>/<string:query>', methods=['GET'])
def search(typeSearch,query):
    print('search')
    try:
        res=searchInput(typeSearch,query)
        return res  
    except:
        raise #TODO handle



if __name__ == '__main__':
    app.run(host='0.0.0.0', port=port)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://10.2.0.2:8000
Press CTRL+C to quit
127.0.0.1 - - [09/Jun/2023 20:44:32] "GET / HTTP/1.1" 200 -


initialize


127.0.0.1 - - [09/Jun/2023 20:45:19] "GET /search/1/?query=hi HTTP/1.1" 404 -
